In [1]:
# importing required libraries
import re
import os
import time
import requests
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('Final.csv')
df

In [ ]:
storyDf = pd.DataFrame()
storyDf['story_urlA'] = df['story_urlA'].values
storyDf['story_urlB'] = df['story_urlB'].values
storyDf

In [ ]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

In [ ]:
# text lowercase
def text_lowercase(text):
    return text.lower()

# remove numbers
def remove_numbers(text):
    result = re.sub(r'\d+', '', text)
    return result

# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# remove whitespace from text
def remove_whitespace(text):
    return  " ".join(text.split())

# remove stopwords function
def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text

input_str = storyDf.story_urlA[0]
print(input_str)

text_lowercase(input_str)
remove_numbers(input_str)
remove_punctuation(input_str)
remove_whitespace(input_str)
remove_stopwords(input_str)

In [ ]:
from numpy import dot
from numpy.linalg import norm

cos_sim = dot(storyDf['story_urlA'], storyDf['story_urlB'])/(norm(storyDf['story_urlA'])*norm(storyDf['story_urlB']))

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# cosine_similarity(storyDf['story_urlA'].values.reshape(1, -1), storyDf['story_urlB'].values.reshape(1, -1))
cosine_similarity(storyDf['story_urlA'].iloc[3], storyDf['story_urlB'].iloc[3])

In [2]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Import your data to a Pandas.DataFrame
df = pd.read_csv('Final.csv')

# Grab the column you'd like to group, filter out duplicate values
# and make sure the values are Unicode
vals = df['story_urlA'].unique().astype('U')


# Write a function for cleaning strings and returning an array of ngrams
def ngrams_analyzer(string):
    string = re.sub(r'[,-./]', r'', string)
    ngrams = zip(*[string[i:] for i in range(5)])  # N-Gram length is 5
    return [''.join(ngram) for ngram in ngrams]

# Construct your vectorizer for building the TF-IDF matrix
vectorizer = TfidfVectorizer(analyzer=ngrams_analyzer)

# Build the matrix!!!
tfidf_matrix = vectorizer.fit_transform(vals)

In [ ]:
!conda install cython

In [4]:
# Import IGN's awesome_cossim_topn module
from sparse_dot_topn import awesome_cossim_topn

# The arguments for awesome_cossim_topn are as follows:
### 1. Our TF-IDF matrix
### 2. Our TF-IDF matrix transposed (allowing us to build a pairwise cosine matrix)
### 3. A top_n filter, which allows us to filter the number of matches returned, which isn't useful for our purposes
### 4. This is our similarity threshold. Only values over 0.8 will be returned
cosine_matrix = awesome_cossim_topn(
  tfidf_matrix,
  tfidf_matrix.transpose(),
  vals.size,
  0.8
)

In [5]:
import re
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sparse_dot_topn import awesome_cossim_topn

# Import your data to a Pandas.DataFrame
df = pd.read_csv('Final.csv')

# Instaniate our lookup hash table
group_lookup = {}


# Write a function for cleaning strings and returning an array of ngrams
def ngrams_analyzer(string):
    string = re.sub(r'[,-./]', r'', string)
    ngrams = zip(*[string[i:] for i in range(5)])  # N-Gram length is 5
    return [''.join(ngram) for ngram in ngrams]


def find_group(row, col):
    # If either the row or the col string have already been given
    # a group, return that group. Otherwise return none
    if row in group_lookup:
        return group_lookup[row]
    elif col in group_lookup:
        return group_lookup[col]
    else:
        return None


def add_vals_to_lookup(group, row, col):
    # Once we know the group name, set it as the value
    # for both strings in the group_lookup
    group_lookup[row] = group
    group_lookup[col] = group


def add_pair_to_lookup(row, col):
    # in this function we'll add both the row and the col to the lookup
    group = find_group(row, col)  # first, see if one has already been added
    if group is not None:
        # if we already know the group, make sure both row and col are in lookup
        add_vals_to_lookup(group, row, col)
    else:
        # if we get here, we need to add a new group.
        # The name is arbitrary, so just make it the row
        add_vals_to_lookup(row, row, col)


# Construct your vectorizer for building the TF-IDF matrix
vectorizer = TfidfVectorizer(analyzer=ngrams_analyzer)

# Grab the column you'd like to group, filter out duplicate values
# and make sure the values are Unicode
vals = df['story_urlA'].unique().astype('U')

# Build the matrix!!!
tfidf_matrix = vectorizer.fit_transform(vals)

cosine_matrix = awesome_cossim_topn(tfidf_matrix, tfidf_matrix.transpose(), vals.size, 0.8)

# Build a coordinate matrix
coo_matrix = cosine_matrix.tocoo()

# for each row and column in coo_matrix
# if they're not the same string add them to the group lookup
for row, col in zip(coo_matrix.row, coo_matrix.col):
    if row != col:
        add_pair_to_lookup(vals[row], vals[col])

df['Group'] = df['story_urlA'].map(group_lookup).fillna(df['story_urlA'])

df.to_csv('data-grouped.csv')

In [ ]:
# Program to measure the similarity between 
# two sentences using cosine similarity.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

df = pd.read_csv('Final.csv')

# X = input("Enter first string: ").lower()
# Y = input("Enter second string: ").lower()
X = df['story_urlA'].iloc[0]
Y = df['story_urlB'].iloc[0]
  
# tokenization
X_list = word_tokenize(X) 
Y_list = word_tokenize(Y)
  
# sw contains the list of stopwords
sw = stopwords.words('english') 
l1 =[];l2 =[]
  
# remove stop words from the string
X_set = {w for w in X_list if not w in sw} 
Y_set = {w for w in Y_list if not w in sw}
  
# form a set containing keywords of both strings 
rvector = X_set.union(Y_set) 
for w in rvector:
    if w in X_set: l1.append(1) # create a vector
    else: l1.append(0)
    if w in Y_set: l2.append(1)
    else: l2.append(0)
c = 0
  
# cosine formula 
for i in range(len(rvector)):
        c+= l1[i]*l2[i]
cosine = c / float((sum(l1)*sum(l2))**0.5)
print("similarity: ", cosine)

In [71]:
def cos_Sim(X, Y):

    # tokenization
    X_list = word_tokenize(X) 
    Y_list = word_tokenize(Y)

    # sw contains the list of stopwords
    sw = stopwords.words('english') 
    l1 =[];l2 =[]

    # remove stop words from the string
    X_set = {w for w in X_list if not w in sw} 
    Y_set = {w for w in Y_list if not w in sw}

    # form a set containing keywords of both strings 
    rvector = X_set.union(Y_set) 
    for w in rvector:
        if w in X_set: l1.append(1) # create a vector
        else: l1.append(0)
        if w in Y_set: l2.append(1)
        else: l2.append(0)
    c = 0

    # cosine formula 
    for i in range(len(rvector)):
            c+= l1[i]*l2[i]
    
    try:
        cosine = c / float((sum(l1)*sum(l2))**0.5)
    except:
        cosine = 0
    
    return cosine

In [75]:
# Program to measure the similarity between 
# two sentences using cosine similarity.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

df = pd.read_csv('Final.csv')

cs_simStory = []
cs_simRisk = []

for i,row in df.iterrows():
    X = df.story_urlA.iloc[i]
    Y = df.story_urlB.iloc[i]
    cosValS = cos_Sim(str(X), str(Y))
    cs_simStory.append(cosValS)
    
    I = df.risk_urlA.iloc[i]
    J = df.risk_urlB.iloc[i]
    cosValR = cos_Sim(str(I), str(J))
    cs_simRisk.append(cosValR)
#     print("similarity: ", cosValS)

In [76]:
df['cs_simStory'] = pd.DataFrame(cs_simStory)
df['cs_simRisk'] = pd.DataFrame(cs_simRisk)

df.to_csv('CS_Output.csv')